**Mount Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Import Libraries**

In [ ]:
import os
import sys
import numpy as np
from numpy import mean,std
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, NuSVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis, LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from lightgbm import LGBMClassifier
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import train_test_split, RepeatedKFold
from sklearn.svm import SVC
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import metrics
import itertools
from itertools import chain, product
from itertools import zip_longest

**Load Datasets**

In [ ]:
Train_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Train.csv')
Test_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Test.csv')
Val_data = pd.read_csv('/content/drive/MyDrive/COVID-19/Constraint_Val.csv')

In [ ]:
Train_data.head()

,id,tweet,label
0,1,The CDC currently reports 99031 deaths. In gen...,real
1,2,States reported 1121 deaths a small rise from ...,real
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real
4,5,Populous states can generate large case counts...,real


In [ ]:
Val_data.head()

,id,tweet,label
0,1,Chinese converting to Islam after realising th...,fake
1,2,11 out of 13 people (from the Diamond Princess...,fake
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real


In [ ]:
Test_data.head()

,id,tweet
0,1,Our daily update is published. States reported...
1,2,Alfalfa is the only cure for COVID-19.
2,3,President Trump Asked What He Would Do If He W...
3,4,States reported 630 deaths. We are still seein...
4,5,This is the sixth time a global health emergen...


In [ ]:
Train_data.shape, Val_data.shape, Test_data.shape

((6420, 3), (2140, 3), (2140, 2))

**Text Preprocessing**

In [ ]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return " ".join(filtered_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
Train_data['cleanText'] = Train_data['tweet'].map(lambda s:preprocess(s))
Val_data['cleanText'] = Val_data['tweet'].map(lambda s:preprocess(s))
Test_data['cleanText'] = Test_data['tweet'].map(lambda s:preprocess(s))

In [ ]:
Train_data.head()

,id,tweet,label,cleanText
0,1,The CDC currently reports 99031 deaths. In gen...,real,cdc currently reports deaths general discrepan...
1,2,States reported 1121 deaths a small rise from ...,real,states reported deaths small rise last tuesday...
2,3,Politically Correct Woman (Almost) Uses Pandem...,fake,politically correct woman almost uses pandemic...
3,4,#IndiaFightsCorona: We have 1524 #COVID testin...,real,indiafightscorona covid testing laboratories i...
4,5,Populous states can generate large case counts...,real,populous states generate large case counts loo...


In [ ]:
Val_data.head()

,id,tweet,label,cleanText
0,1,Chinese converting to Islam after realising th...,fake,chinese converting islam realising muslim affe...
1,2,11 out of 13 people (from the Diamond Princess...,fake,people diamond princess cruise ship intially t...
2,3,"COVID-19 Is Caused By A Bacterium, Not Virus A...",fake,covid caused bacterium virus treated aspirin
3,4,Mike Pence in RNC speech praises Donald Trump’...,fake,mike pence rnc speech praises donald trump cov...
4,5,6/10 Sky's @EdConwaySky explains the latest #C...,real,sky edconwaysky explains latest covid data gov...


In [ ]:
Test_data.head()

,id,tweet,cleanText
0,1,Our daily update is published. States reported...,daily update published states reported tests n...
1,2,Alfalfa is the only cure for COVID-19.,alfalfa cure covid
2,3,President Trump Asked What He Would Do If He W...,president trump asked would catch coronavirus ...
3,4,States reported 630 deaths. We are still seein...,states reported deaths still seeing solid nati...
4,5,This is the sixth time a global health emergen...,sixth time global health emergency declared in...


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer

def nGram(train_text, test_text):
  #unique word and word count
  vectorizer = HashingVectorizer(n_features=2**4)
  
  X1 = vectorizer.fit_transform(train_text)
  #word_list = vectorizer.get_feature_names()
  word_list = vectorizer.get_params(deep=True)
  word_matrix1 = X1.toarray()
  count_list1 = word_matrix1.sum(axis=0)
  A = pd.DataFrame(word_matrix1, columns = word_list)


  X2 = vectorizer.transform(test_text)
  #word_list = vectorizer.get_feature_names()
  word_list = vectorizer.get_params(deep=True)
  word_matrix2 = X2.toarray()
  count_list2 = word_matrix2.sum(axis=0)
  B = pd.DataFrame(word_matrix2, columns = word_list)
  return A, B
  

In [ ]:
X_train, X_test = nGram(Train_data['cleanText'], Val_data['cleanText'])

#y_train = Train_data['label'][0:1000]
#y_test = Val_data['label'][0:1000]

#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

In [ ]:

from sklearn import preprocessing
label_encoder = preprocessing.LabelEncoder()

y_train = label_encoder.fit_transform(Train_data['label'])
y_test = label_encoder.transform(Val_data['label'])
label_encoder.classes_

array(['fake', 'real'], dtype=object)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6420, 16), (2140, 16), (6420,), (2140,))

In [ ]:
X_train

,alternate_sign,analyzer,binary,decode_error,dtype,encoding,input,lowercase,n_features,ngram_range,norm,preprocessor,stop_words,strip_accents,token_pattern,tokenizer
0,0.250000,-0.500000,0.250000,0.000000,0.500000,0.000000,0.000000,0.000000,0.2500,-0.250000,0.000000,0.000000,0.250000,-0.250000,-0.250000,-0.250000
1,0.654654,0.000000,-0.436436,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.436436,0.436436,0.000000,0.000000,0.000000
2,0.353553,0.000000,0.000000,-0.353553,0.353553,0.353553,0.000000,0.000000,0.0000,0.353553,0.353553,-0.353553,0.000000,-0.353553,0.000000,0.000000
3,0.000000,-0.408248,0.000000,0.000000,0.000000,-0.408248,0.408248,0.000000,0.0000,0.408248,0.408248,0.408248,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,-0.426401,0.213201,-0.213201,0.000000,0.000000,0.639602,0.0000,0.000000,-0.426401,0.000000,0.000000,0.213201,0.213201,0.213201
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6415,-0.377964,0.377964,0.000000,0.000000,0.000000,0.000000,0.377964,-0.377964,0.0000,0.377964,0.000000,0.377964,-0.377964,0.000000,0.000000,0.000000
6416,0.242536,-0.242536,0.000000,0.242536,0.242536,-0.242536,0.000000,-0.242536,0.0000,0.485071,-0.242536,0.485071,-0.242536,0.000000,0.242536,0.000000
6417,-0.277350,0.277350,0.000000,0.000000,0.277350,0.000000,0.000000,-0.277350,-0.5547,0.554700,0.000000,0.000000,0.000000,0.000000,0.277350,0.000000
6418,0.000000,0.408248,0.000000,0.408248,0.000000,0.408248,0.408248,0.000000,0.0000,0.408248,0.000000,0.408248,0.000000,0.000000,0.000000,0.000000


In [ ]:
X_test

,alternate_sign,analyzer,binary,decode_error,dtype,encoding,input,lowercase,n_features,ngram_range,norm,preprocessor,stop_words,strip_accents,token_pattern,tokenizer
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.447214,0.447214,0.000000,0.000000,0.000000,-0.447214,0.000000,0.000000,-0.447214,0.447214
1,0.000000,0.458831,-0.229416,0.000000,0.229416,0.000000,-0.688247,0.000000,0.229416,0.229416,0.000000,0.000000,-0.229416,0.000000,0.229416,0.000000
2,-0.353553,0.000000,0.000000,0.000000,0.000000,0.000000,-0.353553,0.353553,0.000000,0.707107,0.000000,0.000000,0.353553,0.000000,0.000000,0.000000
3,0.000000,-0.250000,0.000000,0.000000,-0.250000,0.250000,0.500000,-0.250000,0.000000,0.250000,0.000000,0.500000,0.250000,0.250000,0.250000,0.000000
4,0.000000,0.000000,-0.301511,0.603023,0.000000,0.000000,0.000000,-0.603023,0.000000,0.301511,0.000000,0.000000,0.000000,0.000000,0.301511,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2135,-0.174078,0.000000,-0.174078,-0.348155,0.174078,0.000000,0.000000,0.522233,-0.174078,0.174078,-0.348155,0.522233,0.000000,0.174078,0.000000,0.174078
2136,-0.164399,0.000000,0.000000,0.000000,0.000000,0.000000,0.164399,0.000000,-0.493197,0.328798,-0.164399,-0.328798,0.000000,-0.493197,-0.328798,0.328798
2137,0.000000,0.000000,0.000000,0.000000,0.485071,-0.242536,0.000000,0.242536,-0.242536,0.485071,0.000000,-0.485071,0.000000,-0.242536,-0.242536,0.000000
2138,0.000000,0.000000,-0.333333,0.000000,-0.333333,0.666667,0.000000,0.000000,0.000000,0.333333,0.000000,-0.333333,0.000000,-0.333333,0.000000,0.000000


In [ ]:
y_train

array([1, 1, 0, ..., 0, 0, 1])

In [ ]:
y_test

array([0, 0, 0, ..., 0, 0, 1])

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((6420, 16), (2140, 16), (6420,), (2140,))

**Model Evaluation**

In [ ]:
dict_classifiers = {
    "Nearest Neighbors":KNeighborsClassifier(3),
    "LogisticRegression":LogisticRegression(),
    #"NuSVC":NuSVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(max_depth=5),
    "Random Forest": RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    "MLP": MLPClassifier(alpha=1, max_iter=1000),
    "AdaBoost":AdaBoostClassifier(),
    "GaussianNB":GaussianNB(),
    "LinearDiscriminant":LinearDiscriminantAnalysis(),
    "QuadraticDiscriminant":QuadraticDiscriminantAnalysis(),
    "GradientBoost":GradientBoostingClassifier(),
    #"MultinomialNB":MultinomialNB(),
    "SGD":SGDClassifier(),
    "LGBM":LGBMClassifier(),
    #"XGB":XGBClassifier(),
    "Gaussian Process Classifier":GaussianProcessClassifier(1.0 * RBF(1.0)),
    "Support Vector Machine 1":SVC(kernel="linear", C=0.025),
    "Support Vector Machine 2":SVC(gamma=2, C=1)
}

In [ ]:
def evaluation_matrix(model, testX, testy):
    # predict probabilities for test set
    yhat_probs = model.predict(testX)
    #print(yhat_probs)
    # predict crisp classes for test set
    #yhat_classes = model.predict_classes(testX)
    # reduce to 1d array
    #yhat_probs = yhat_probs[:, 0]
    #print(yhat_probs)
    #yhat_classes = yhat_classes
    yhat_classes = yhat_probs
    #accuracy: (tp + tn) / (p + n)
    accuracy = accuracy_score(testy, yhat_classes)
    print('Accuracy: %f' % accuracy)
    # precision tp / (tp + fp)
    precision = precision_score(testy, yhat_classes, average='weighted')
    print('Precision: %f' % precision)
    # recall: tp / (tp + fn)
    recall = recall_score(testy, yhat_classes, average='weighted')
    print('Recall: %f' % recall)
    # f1: 2 tp / (2 tp + fp + fn)
    f1 = f1_score(testy, yhat_classes, average='weighted')
    print('F1 score: %f' % f1)
    # kappa
    kappa = cohen_kappa_score(testy, yhat_classes)
    print('Cohens kappa: %f' % kappa)
    # ROC AUC
    #auc = roc_auc_score(testy, yhat_probs, multi_class = 'ovr')
    #print('ROC AUC: %f' % auc)
    #confusion matrix
    matrix = confusion_matrix(testy, yhat_classes)
    print(matrix)
    return accuracy, precision, recall, f1, kappa

In [ ]:
def batch_classify(dict_classifiers, X_train,X_test,y_train,y_test):
    no_classifiers = len(dict_classifiers)
    dict_super_models = {}

    for classifier_name,classifier in list(dict_classifiers.items())[:no_classifiers]:
        print(classifier_name)
        classifier.fit(X_train, y_train)
        train_score = classifier.score(X_train, y_train)
        test_score = classifier.score(X_test, y_test)
        accuracy, precision, recall, f1, kappa = evaluation_matrix(classifier, X_test, y_test)
        y_pred = classifier.predict(X_test)
        print('train score:{%0.2f} test_score:{%0.2f}' %(train_score, test_score))
        dict_super_models[classifier_name] = {'Supervised': classifier_name, 'Accuracy': accuracy, 'Precision': precision, 'Recall': recall, 'F1': f1, 'Kappa': kappa}   

    return dict_super_models

In [ ]:
dict_super_models = batch_classify(dict_classifiers, X_train,X_test,y_train,y_test)

Nearest Neighbors
Accuracy: 0.629439
Precision: 0.630671
Recall: 0.629439
F1 score: 0.629640
Cohens kappa: 0.259040
[[651 369]
 [424 696]]
train score:{0.82} test_score:{0.63}
LogisticRegression
Accuracy: 0.581776
Precision: 0.580880
Recall: 0.581776
F1 score: 0.580790
Cohens kappa: 0.159329
[[540 480]
 [415 705]]
train score:{0.59} test_score:{0.58}
Decision Tree
Accuracy: 0.615421
Precision: 0.614727
Recall: 0.615421
F1 score: 0.613802
Cohens kappa: 0.225871
[[560 460]
 [363 757]]
train score:{0.66} test_score:{0.62}
Random Forest
Accuracy: 0.637383
Precision: 0.640288
Recall: 0.637383
F1 score: 0.631227
Cohens kappa: 0.265788
[[517 503]
 [273 847]]
train score:{0.69} test_score:{0.64}
MLP
Accuracy: 0.605607
Precision: 0.605062
Recall: 0.605607
F1 score: 0.605123
Cohens kappa: 0.208102
[[578 442]
 [402 718]]
train score:{0.62} test_score:{0.61}
AdaBoost
Accuracy: 0.666355
Precision: 0.666516
Recall: 0.666355
F1 score: 0.666421
Cohens kappa: 0.331543
[[668 352]
 [362 758]]
train score

In [ ]:
def display_dict_models(dict_models, sort_by='Accuracy'):
#def display_dict_models(dict_models, sort_by='train_score'):
    cls = [key for key in dict_models.keys()]
    accuracy = [dict_models[key]['Accuracy'] for key in cls]
    precision = [dict_models[key]['Precision'] for key in cls]
    recall = [dict_models[key]['Recall'] for key in cls]
    f1 = [dict_models[key]['F1'] for key in cls]
    kappa = [dict_models[key]['Kappa'] for key in cls]
    #training_t = [dict_models[key]['train_time'] for key in cls]
    
    #df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),4)), columns = ['classifier', 'train_score', 'test_score', 'train_time'])
    df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),6)), columns = ['classifier', 'Accuracy', 'Precision', 'Recall', 'F1', 'Kappa'])
    # df_ = pd.DataFrame(data=np.zeros(shape=(len(cls),3)), columns = ['classifier', 'train_score', 'train_time'])
    for ii in range(0,len(cls)):
        df_.loc[ii, 'classifier'] = cls[ii]
        df_.loc[ii, 'Accuracy'] = accuracy[ii]
        df_.loc[ii, 'Precision'] = precision[ii]
        df_.loc[ii, 'Recall'] = recall[ii]
        df_.loc[ii, 'F1'] = f1[ii]
        df_.loc[ii, 'Kappa'] = kappa[ii]
       
        #df_.loc[ii, 'train_time'] = training_t[ii]
    
    display(df_.sort_values(by=sort_by, ascending=False))
    return df_

In [ ]:
df = display_dict_models(dict_super_models)

,classifier,Accuracy,Precision,Recall,F1,Kappa
11,LGBM,0.679439,0.679237,0.679439,0.679290,0.356969
9,GradientBoost,0.668224,0.668614,0.668224,0.668349,0.335636
5,AdaBoost,0.666355,0.666516,0.666355,0.666421,0.331543
14,Support Vector Machine 2,0.650935,0.653036,0.650935,0.651052,0.302813
12,Gaussian Process Classifier,0.649065,0.649364,0.649065,0.649174,0.297118
3,Random Forest,0.637383,0.640288,0.637383,0.631227,0.265788
0,Nearest Neighbors,0.629439,0.630671,0.629439,0.629640,0.259040
2,Decision Tree,0.615421,0.614727,0.615421,0.613802,0.225871
8,QuadraticDiscriminant,0.609346,0.610546,0.609346,0.609558,0.218828
6,GaussianNB,0.608411,0.609347,0.608411,0.608622,0.216617


In [ ]:
print(df.to_latex(index=False))

\begin{tabular}{lrrrrr}
\toprule
                  classifier &  Accuracy &  Precision &    Recall &        F1 &     Kappa \\
\midrule
           Nearest Neighbors &  0.629439 &   0.630671 &  0.629439 &  0.629640 &  0.259040 \\
          LogisticRegression &  0.581776 &   0.580880 &  0.581776 &  0.580790 &  0.159329 \\
               Decision Tree &  0.615421 &   0.614727 &  0.615421 &  0.613802 &  0.225871 \\
               Random Forest &  0.637383 &   0.640288 &  0.637383 &  0.631227 &  0.265788 \\
                         MLP &  0.605607 &   0.605062 &  0.605607 &  0.605123 &  0.208102 \\
                    AdaBoost &  0.666355 &   0.666516 &  0.666355 &  0.666421 &  0.331543 \\
                  GaussianNB &  0.608411 &   0.609347 &  0.608411 &  0.608622 &  0.216617 \\
          LinearDiscriminant &  0.581776 &   0.580899 &  0.581776 &  0.580832 &  0.159403 \\
       QuadraticDiscriminant &  0.609346 &   0.610546 &  0.609346 &  0.609558 &  0.218828 \\
               GradientBoost